In [1]:
# BiLSTM

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Flatten
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
file_path = 'data/statcast.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Fill missing values with 0
data = data.fillna(0)

if 'pitch_hand' in data:
    data = pd.get_dummies(data, columns=['pitch_hand'], drop_first=True)

# Extract player_ids that exist in 2015, 2016, 2017, 2018, and 2019
data_2015 = data[data['year'] == 2015]
data_2016 = data[data['year'] == 2016]
data_2017 = data[data['year'] == 2017]
data_2018 = data[data['year'] == 2018]
data_2019 = data[data['year'] == 2019]

player_ids_2015 = set(data_2015['player_id'].unique())
player_ids_2016 = set(data_2016['player_id'].unique())
player_ids_2017 = set(data_2017['player_id'].unique())
player_ids_2018 = set(data_2018['player_id'].unique())
player_ids_2019 = set(data_2019['player_id'].unique())

common_player_ids = player_ids_2015 & player_ids_2016 & player_ids_2017 & player_ids_2018 & player_ids_2019

# Extract data corresponding to common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2015, 2016, 2017, and 2018
final = common_data[common_data['year'].isin([2015, 2016, 2017, 2018])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Split independent and dependent variables
X = final[features].values
y = final[target].values

# Scale the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Function to create sequences for time series data
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 4  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data
X_train, y_train = X_seq, y_seq

# Initialize lists to store metrics
iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # Initialize the BiLSTM model
    model_BiLSTM = Sequential()
    model_BiLSTM.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=(seq_length, X_train.shape[2])))
    model_BiLSTM.add(Bidirectional(LSTM(64, return_sequences=True)))
    model_BiLSTM.add(Dropout(rate=0.5))
    model_BiLSTM.add(Flatten())
    model_BiLSTM.add(Dense(512, activation="relu"))
    model_BiLSTM.add(Dropout(rate=0.5))
    model_BiLSTM.add(Dense(64, activation="relu"))
    model_BiLSTM.add(Dense(1, activation='relu'))

    # Compile the model
    adam = optimizers.Adam(learning_rate=0.001)
    model_BiLSTM.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_BiLSTM = model_BiLSTM.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=0, callbacks=[early_stopping])

    # Filter 2019 data
    data_19 = common_data[common_data['year'] == 2019]

    # Scale 2019 data
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # Function to create sequences for prediction
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # Predict 2019 data
    y_pred_scaled_BiLSTM = model_BiLSTM.predict(X_2019_seq)

    # Inverse scale the predictions
    y_pred_BiLSTM = scaler_y.inverse_transform(y_pred_scaled_BiLSTM)

    # Actual 2019 p_era values
    y_test_actual = data_19[target].values

    # Calculate RMSE
    rmse_BiLSTM = np.sqrt(mean_squared_error(y_test_actual, y_pred_BiLSTM))
    rmse_list.append(rmse_BiLSTM)

    # Calculate MAE
    mae_BiLSTM = mean_absolute_error(y_test_actual, y_pred_BiLSTM)
    mae_list.append(mae_BiLSTM)
    
    # Calculate MAPE
    mape_BiLSTM = mean_absolute_percentage_error(y_test_actual, y_pred_BiLSTM)
    mape_list.append(mape_BiLSTM)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_BiLSTM}, MAE: {mae_BiLSTM}, MAPE: {mape_BiLSTM}')

# Calculate average metrics
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

4/4 [==============================] - 4s 5ms/step
Iteration 1/5 - RMSE: 0.9412054359758849, MAE: 0.7064478975985229, MAPE: 0.15660813596255518
4/4 [==============================] - 3s 5ms/step
Iteration 2/5 - RMSE: 0.8623552279569364, MAE: 0.6531005824113092, MAPE: 0.15260372539601463
4/4 [==============================] - 4s 6ms/step
Iteration 3/5 - RMSE: 0.6909087817821985, MAE: 0.5413881567947003, MAPE: 0.12544157062429093
4/4 [==============================] - 6s 7ms/step
Iteration 4/5 - RMSE: 0.8548148874758793, MAE: 0.6681756834623193, MAPE: 0.15462536338983482
4/4 [==============================] - 5s 7ms/step
Iteration 5/5 - RMSE: 0.806702604249327, MAE: 0.6167536115646363, MAPE: 0.1374021953053235
Average RMSE: 0.8311973874880451
Average MAE: 0.6371731863662976
Average MAPE: 0.1453361981356038


In [2]:
# CNN-BiLSTM

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, Reshape
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = 'data/statcast.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Fill missing values with 0
data = data.fillna(0)

if 'pitch_hand' in data:
    data = pd.get_dummies(data, columns=['pitch_hand'], drop_first=True)

# Extract player_ids that exist in 2015, 2016, 2017, 2018, and 2019
data_2015 = data[data['year'] == 2015]
data_2016 = data[data['year'] == 2016]
data_2017 = data[data['year'] == 2017]
data_2018 = data[data['year'] == 2018]
data_2019 = data[data['year'] == 2019]

player_ids_2015 = set(data_2015['player_id'].unique())
player_ids_2016 = set(data_2016['player_id'].unique())
player_ids_2017 = set(data_2017['player_id'].unique())
player_ids_2018 = set(data_2018['player_id'].unique())
player_ids_2019 = set(data_2019['player_id'].unique())

common_player_ids = player_ids_2015 & player_ids_2016 & player_ids_2017 & player_ids_2018 & player_ids_2019

# Extract data corresponding to common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data corresponding to the years 2015, 2016, 2017, and 2018
final = common_data[common_data['year'].isin([2015, 2016, 2017, 2018])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Split independent and dependent variables
X = final[features].values
y = final[target].values

# Scale the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Function to create sequences for time series data
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 4  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data
X_train, y_train = X_seq, y_seq

# Initialize lists to store metrics
iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # Initialize the CNN-BiLSTM model
    model_CNN_BiLSTM = Sequential()
    model_CNN_BiLSTM.add(Conv1D(filters=64, kernel_size=4, activation='relu', input_shape=(seq_length, X_train.shape[2])))
    model_CNN_BiLSTM.add(Flatten())
    model_CNN_BiLSTM.add(Dense(64, activation='relu'))
    model_CNN_BiLSTM.add(Reshape((1, 64)))
    model_CNN_BiLSTM.add(Bidirectional(LSTM(64, return_sequences=True)))
    model_CNN_BiLSTM.add(Bidirectional(LSTM(64)))
    model_CNN_BiLSTM.add(Dense(1))

    # Compile the model
    adam = optimizers.Adam(learning_rate=0.001)
    model_CNN_BiLSTM.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_CNN_BiLSTM = model_CNN_BiLSTM.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=0, callbacks=[early_stopping])

    # Filter 2019 data
    data_19 = common_data[common_data['year'] == 2019]

    # Scale 2019 data
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # Function to create sequences for prediction
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # Predict 2019 data
    y_pred_scaled_CNN_BiLSTM = model_CNN_BiLSTM.predict(X_2019_seq)

    # Inverse scale the predictions
    y_pred_CNN_BiLSTM = scaler_y.inverse_transform(y_pred_scaled_CNN_BiLSTM)

    # Actual 2019 p_era values
    y_test_actual = data_19[target].values

    # Calculate RMSE
    rmse_CNN_BiLSTM = np.sqrt(mean_squared_error(y_test_actual, y_pred_CNN_BiLSTM))
    rmse_list.append(rmse_CNN_BiLSTM)

    # Calculate MAE
    mae_CNN_BiLSTM = mean_absolute_error(y_test_actual, y_pred_CNN_BiLSTM)
    mae_list.append(mae_CNN_BiLSTM)
    
    # Calculate MAPE
    mape_CNN_BiLSTM = mean_absolute_percentage_error(y_test_actual, y_pred_CNN_BiLSTM)
    mape_list.append(mape_CNN_BiLSTM)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_CNN_BiLSTM}, MAE: {mae_CNN_BiLSTM}, MAPE: {mape_CNN_BiLSTM}')

# Calculate average metrics
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

4/4 [==============================] - 4s 5ms/step
Iteration 1/5 - RMSE: 0.8065517511945001, MAE: 0.6246055731252461, MAPE: 0.14940905723891118
4/4 [==============================] - 5s 4ms/step
Iteration 2/5 - RMSE: 0.8686547528440962, MAE: 0.6707995301735501, MAPE: 0.15570117602250416
4/4 [==============================] - 9s 5ms/step
Iteration 3/5 - RMSE: 0.7927439798137556, MAE: 0.5976601560576622, MAPE: 0.13860703409978353
4/4 [==============================] - 7s 4ms/step
Iteration 4/5 - RMSE: 0.8945107051126167, MAE: 0.6923840940699857, MAPE: 0.1622491101339976
4/4 [==============================] - 9s 5ms/step
Iteration 5/5 - RMSE: 0.8916029911486512, MAE: 0.6772230600709673, MAPE: 0.15390812394711076
Average RMSE: 0.850812836022724
Average MAE: 0.6525344826994823
Average MAPE: 0.15197490028846145


In [3]:
# BiLSTM-ED

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Flatten, RepeatVector, TimeDistributed
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
file_path = 'data/statcast.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Fill missing values with 0
data = data.fillna(0)

if 'pitch_hand' in data:
    data = pd.get_dummies(data, columns=['pitch_hand'], drop_first=True)
# Extract player_ids that exist in 2015, 2016, 2017, 2018, and 2019
data_2015 = data[data['year'] == 2015]
data_2016 = data[data['year'] == 2016]
data_2017 = data[data['year'] == 2017]
data_2018 = data[data['year'] == 2018]
data_2019 = data[data['year'] == 2019]

player_ids_2015 = set(data_2015['player_id'].unique())
player_ids_2016 = set(data_2016['player_id'].unique())
player_ids_2017 = set(data_2017['player_id'].unique())
player_ids_2018 = set(data_2018['player_id'].unique())
player_ids_2019 = set(data_2019['player_id'].unique())

common_player_ids = player_ids_2015 & player_ids_2016 & player_ids_2017 & player_ids_2018 & player_ids_2019

# Extract data corresponding to common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data corresponding to the years 2015, 2016, 2017, and 2018
final = common_data[common_data['year'].isin([2015, 2016, 2017, 2018])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Split independent and dependent variables
X = final[features].values
y = final[target].values

# Scale the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Function to create sequences for time series data
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 4  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data
X_train, y_train = X_seq, y_seq

# Initialize lists to store metrics
iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # Initialize the BiLSTM-ED model
    model_BiLSTM_ED = Sequential()
    model_BiLSTM_ED.add(Bidirectional(LSTM(64, return_sequences=False), input_shape=(seq_length, X_train.shape[2])))
    model_BiLSTM_ED.add(RepeatVector(seq_length))
    model_BiLSTM_ED.add(Bidirectional(LSTM(64, return_sequences=True)))
    model_BiLSTM_ED.add(TimeDistributed(Dense(1)))

    # Compile the model
    adam = optimizers.Adam(learning_rate=0.001)
    model_BiLSTM_ED.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_BiLSTM_ED = model_BiLSTM_ED.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=0, callbacks=[early_stopping])

    # Filter 2019 data
    data_19 = common_data[common_data['year'] == 2019]

    # Scale 2019 data
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # Function to create sequences for prediction
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # Predict 2019 data
    y_pred_scaled_BiLSTM_ED = model_BiLSTM_ED.predict(X_2019_seq)

    # Inverse scale the predictions
    y_pred_BiLSTM_ED = scaler_y.inverse_transform(y_pred_scaled_BiLSTM_ED[:, -1, :])  # Take the last time step

    # Actual 2019 p_era values
    y_test_actual = data_19[target].values

    # Calculate RMSE
    rmse_BiLSTM_ED = np.sqrt(mean_squared_error(y_test_actual, y_pred_BiLSTM_ED))
    rmse_list.append(rmse_BiLSTM_ED)

    # Calculate MAE
    mae_BiLSTM_ED = mean_absolute_error(y_test_actual, y_pred_BiLSTM_ED)
    mae_list.append(mae_BiLSTM_ED)
    
    # Calculate MAPE
    mape_BiLSTM_ED = mean_absolute_percentage_error(y_test_actual, y_pred_BiLSTM_ED)
    mape_list.append(mape_BiLSTM_ED)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_BiLSTM_ED}, MAE: {mae_BiLSTM_ED}, MAPE: {mape_BiLSTM_ED}')

# Calculate average metrics
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

4/4 [==============================] - 10s 6ms/step
Iteration 1/5 - RMSE: 0.6844446849027701, MAE: 0.5183175593464314, MAPE: 0.11990969998030102
4/4 [==============================] - 24s 9ms/step
Iteration 2/5 - RMSE: 0.7104698971360792, MAE: 0.5425713931812959, MAPE: 0.12738539976202248
4/4 [==============================] - 10s 6ms/step
Iteration 3/5 - RMSE: 0.6845238829238423, MAE: 0.5169340154904277, MAPE: 0.12044582822707123
4/4 [==============================] - 11s 6ms/step
Iteration 4/5 - RMSE: 0.6931236686723767, MAE: 0.5197071018539557, MAPE: 0.11879152747371174
4/4 [==============================] - 12s 6ms/step
Iteration 5/5 - RMSE: 0.6909088590368059, MAE: 0.5115442158194149, MAPE: 0.12025878946791248
Average RMSE: 0.6926941985343749
Average MAE: 0.5218148571383051
Average MAPE: 0.12135824898220379
